# Lake Levels ETL
## Setting up db tables
---

In [4]:
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import Column, Date, Integer, String, Float, Table

# import schema from python file
from schema_lake_levels import Base
from schema_lake_levels import Lake_names, Lake_levels

### Create tables from schema file in SQLite

In [5]:
# create sqlite file if not exist, establish connection, create tables from schema (using 'Base')
database_path = "resources/lake-levels-data.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
Base.metadata.create_all(engine)

### Entering tables into SQLite

#### Lake names, ids --> reformat, enter into database

In [6]:
# read in csv file
lake_id_df = pd.read_csv("data/water-level/lakes/MCWD_Lake_ID.csv")

lake_id_df.head()

,LAKE_NAME,LAKE_ID,latitude,longitude
0,Hiawatha,27001800,44.921034,-93.236141
1,Mother,27002300,44.893298,-93.241013
2,Nokomis,27001900,44.908634,-93.242187
3,Taft,27068300,44.892951,-93.249752
4,Legion,27002400,44.885760,-93.262240


In [7]:
# make col names lowercase
lake_id_df.columns = ['name', 'id', 'lat', 'lng']

# drop duplicate ids
lake_id_df.drop_duplicates(subset='id', inplace=True)

# set 'lake_id' as index
lake_id_df.set_index('id', inplace=True)

# preview
lake_id_df.head()

,name,lat,lng
id,,,
27001800,Hiawatha,44.921034,-93.236141
27002300,Mother,44.893298,-93.241013
27001900,Nokomis,44.908634,-93.242187
27068300,Taft,44.892951,-93.249752
27002400,Legion,44.885760,-93.262240


In [8]:
# export lake_id_df as SQL table 'lake_names'
# lake_id_df.to_sql('lake_names', con=engine, if_exists='replace', index=True)
print("Values inserted into: 'lake_names'")

Values inserted into: 'lake_names'


In [142]:
# function to convert all lake data for a given lake id into dict format
def lake_to_dict(lake_id):

    # grab all measurements for lake
    elevations = list(test.loc[test.id == lake_id].elevation)
    read_dates = list(test.loc[test.id == lake_id].read_date)
    datum_adjs = list(test.loc[test.id == lake_id].datum_adj)
    
    measurements = list(zip(elevations, read_dates, datum_adjs))

    measurement_keys = ['elevation', 'read_date', 'datum_adj']
    
    # make measurements json format
    measurements_json = []
    for i in range(len(aslist)):
        measurement_dict = dict(zip(measurement_keys, aslist[i]))
        measurements_json.append(measurement_dict)

    # create lake_dict using measurements_json
    lake_dict = {
        'lake':
            {
                'name': lake_id_df['name'][lake_id],
                'id': lake_id,
                'location': {
                    'lat': lake_id_df['lat'][lake_id],
                    'lng': lake_id_df['lng'][lake_id]
                },
                'measurements': measurements_json
            }
    }
    
    return lake_dict

#### Scrape lake level data for all lakes in watershed, reformat and enter into SQLite

In [9]:
# base url for scraping lake-level data
base_url = "https://files.dnr.state.mn.us/cgi-bin/lk_levels_dump.pl?format=csv&id="

In [17]:
# variable to track progress of loop
lake_count = 0

# store ids for lakes that failed and succeeded to be inserted
failed = []
successful = []

df_list = []

# loop through index of lake_id_df (lake ids are the index)
for lake_id in lake_id_df.index:
    
    lake_count += 1
    print(f"Getting data for lake {lake_count} of {len(lake_id_df.index)}.......")
    print(f"Lake id: {lake_id}.........")
    
    
    try:
        # read data from url into df
        df = pd.read_csv(f"{base_url}{lake_id}")

        # change column names to lowercase
        df.columns = map(str.lower, df.columns)

        # rename 'chr_id' to 'id'
        df.rename(columns={"chr_id": "id"}, inplace=True)

        # drop duplicate date entries
        df.drop_duplicates(subset='read_date', inplace=True)
        
        df_list.append(df.copy())
        
        # set multi-index for multiple primary keys
        df.set_index(['id', 'read_date'], inplace=True)
        
        
        
        # insert df into sql table 'lake_levels'
#         df.to_sql('lake_levels', con=engine, if_exists='append', index=True, index_label=['id', 'read_date'])
        print("..............Values successfully inserted into 'lake_levels'")
        print("------------------------------------------------------------------\n\n")
        successful.append(lake_id)
    except:
        print(".......................Process failed")
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n\n")
        failed.append(lake_id)

Getting data for lake 1 of 141.......
Lake id: 27001800.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 2 of 141.......
Lake id: 27002300.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 3 of 141.......
Lake id: 27001900.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 4 of 141.......
Lake id: 27068300.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 5 of 141.......
Lake id: 27002400.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lak

..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 44 of 141.......
Lake id: 27009500.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 45 of 141.......
Lake id: 27010800.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 46 of 141.......
Lake id: 27046800.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 47 of 141.......
Lake id: 27008600.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 48 of 141.......
Lake id: 27082200.........
..............V

..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 86 of 141.......
Lake id: 27051800.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 87 of 141.......
Lake id: 27013313.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 88 of 141.......
Lake id: 27052200.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 89 of 141.......
Lake id: 10020600.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 90 of 141.......
Lake id: 10001100.........
..............V

..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 128 of 141.......
Lake id: 27018300.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 129 of 141.......
Lake id: 10004900.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 130 of 141.......
Lake id: 10013900.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 131 of 141.......
Lake id: 27095800.........
..............Values successfully inserted into 'lake_levels'
------------------------------------------------------------------


Getting data for lake 132 of 141.......
Lake id: 27093900.........
..........

In [18]:
# print failed lake additions
print(f"Failed additions: {len(failed)}")
print("----------------------------------------------\n")
for id in failed:
    print(id, lake_id_df.name[id])

Failed additions: 0
----------------------------------------------



In [9]:
# print successful additions
print(f"Successful additions: {len(successful)}")
print("----------------------------------------------\n")
for id in successful:
    print(id, lake_id_df.name[id])

Successful additions: 141
----------------------------------------------

27001800 Hiawatha
27002300 Mother
27001900 Nokomis
27068300 Taft
27002400 Legion
27002200 Diamond
27068400 Milner Pond
27068500 Norby's Pond
27001600 Harriet
27001700 Cemetery
27004000 Lake of the Isles
27003100 Calhoun
27003900 Cedar
27003800 Brownie
27067500 Pamela Pond
27001500 Bass
27065600 Twin
27066400 Wolfe Park
27004100 Edina Mill Pond
27067000 Harvey
27066900 Melody
27005400 Meadowbrook
27066100 South Oak
27071000 Lamplighter
27005100 Victoria
27077900 Unnamed
27071400 Westling
27005300 Unnamed (Cobblecrest)
27005200 Hannan
27071300 Unnamed (Cedar Manor)
27008400 Minnehaha Marsh
27008200 Windsor
27077100 Unnamed
27075100 Unnamed
27075000 Unnamed
27074700 Unnamed
27073900 Unnamed
27013300 Gray's Bay Outlet
27074600 Unnamed
27068701 Unnamed (East)
27013301 Grays Bay
27008500 Libbs
99001038 Unnamed
27009500 Gleason
27010800 Snyder
27046800 Unnamed
27008600 Shaver
27082200 Unnamed
27082400 Unnamed
27013302 W

In [19]:
lake_levels_df = pd.concat(df_list)

In [20]:
lake_levels_df.head()

,id,elevation,read_date,datum_adj
0,27001800,811.40,1926-04-15,NGVD 29
1,27001800,815.35,1926-08-05,NGVD 29
2,27001800,812.72,1927-03-29,NGVD 29
3,27001800,813.04,1927-11-30,NGVD 29
4,27001800,814.50,1928-04-12,NGVD 29


In [86]:
df1 = lake_levels_df.copy()
df2 = lake_id_df.copy()

In [93]:
df = pd.merge(df1, df2, how='left', on='id')
df.head()

,id,elevation,read_date,datum_adj,name,lat,lng
0,27001800,811.40,1926-04-15,NGVD 29,Hiawatha,44.921034,-93.236141
1,27001800,815.35,1926-08-05,NGVD 29,Hiawatha,44.921034,-93.236141
2,27001800,812.72,1927-03-29,NGVD 29,Hiawatha,44.921034,-93.236141
3,27001800,813.04,1927-11-30,NGVD 29,Hiawatha,44.921034,-93.236141
4,27001800,814.50,1928-04-12,NGVD 29,Hiawatha,44.921034,-93.236141


In [96]:
# empty list for storing ids of dropped lakes
dropped = []

print("Dropping lakes with fewer then 10 measurements....")
for lake_id in lake_id_df.index:
    
    # drop lakes with less than 10 measurements
    if len(df.loc[df['id'] == lake_id]['id']) < 10:
        df.drop(df[df['id'] == lake_id].index, inplace=True)
        dropped.append(lake_id)

print(f"Number of laked dropped: {len(dropped)}.")

# reset index
df = df.reset_index(drop=True)

Dropping lakes with fewer then 10 measurements....
Number of laked dropped: 100.


In [97]:
df.head()

,id,elevation,read_date,datum_adj,name,lat,lng
0,27001800,811.40,1926-04-15,NGVD 29,Hiawatha,44.921034,-93.236141
1,27001800,815.35,1926-08-05,NGVD 29,Hiawatha,44.921034,-93.236141
2,27001800,812.72,1927-03-29,NGVD 29,Hiawatha,44.921034,-93.236141
3,27001800,813.04,1927-11-30,NGVD 29,Hiawatha,44.921034,-93.236141
4,27001800,814.50,1928-04-12,NGVD 29,Hiawatha,44.921034,-93.236141


In [145]:
# format all lake data as JSON
lakes_json = []
for lake_id in lake_id_df.index:
    lake_dict = lake_to_dict(lake_id)
    lakes_json.append(lake_dict)

In [157]:
lakes_json[140]

{'lake': {'name': 'Unnamed',
  'id': 10014300,
  'location': {'lat': 44.87818129, 'lng': -93.74894941},
  'measurements': [{'elevation': 811.4,
    'read_date': '1926-04-15',
    'datum_adj': 'NGVD 29'},
   {'elevation': 815.35, 'read_date': '1926-08-05', 'datum_adj': 'NGVD 29'},
   {'elevation': 812.72, 'read_date': '1927-03-29', 'datum_adj': 'NGVD 29'},
   {'elevation': 813.04, 'read_date': '1927-11-30', 'datum_adj': 'NGVD 29'},
   {'elevation': 814.5, 'read_date': '1928-04-12', 'datum_adj': 'NGVD 29'},
   {'elevation': 812.76, 'read_date': '1928-11-28', 'datum_adj': 'NGVD 29'},
   {'elevation': 814.1, 'read_date': '1929-04-21', 'datum_adj': 'NGVD 29'},
   {'elevation': 813.38, 'read_date': '1930-04-01', 'datum_adj': 'NGVD 29'},
   {'elevation': 803.35, 'read_date': '1930-08-26', 'datum_adj': 'NGVD 29'},
   {'elevation': 804.45, 'read_date': '1930-11-17', 'datum_adj': 'NGVD 29'},
   {'elevation': 806.58, 'read_date': '1931-04-01', 'datum_adj': 'NGVD 29'},
   {'elevation': 806.55, 're